<a href="https://colab.research.google.com/github/mazarimono/pyconapac2023/blob/main/panel_watch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## データの観察を工夫して行う

- panelを使って、インタラクティブに作る
- panelを使うメリット。簡単にJupyter上でインタラクティブに動作する可視化アプリを作れる
    - JupyterDash: Reactの勉強にはよいけど、コード量が多め
    - streamlit-jupyter: まだ出たばっかりだしなぁ・・・
    - panel: numfocusだし、Jupyter上で動かすとなると、現状はファーストチョイスかなぁ

In [3]:
import pandas as pd
import plotly.express as px
import panel as pn
from pathlib import Path
pn.extension('plotly')

In [4]:
# データの読み込み
d_path = 'https://raw.githubusercontent.com/mazarimono/pyconapac2023/main/data/all_data.csv'
df = pd.read_csv(d_path, index_col=0)
df.index = pd.to_datetime(df.index)
df['year'] = df.index.map(lambda x: x.year)
cls_list = ['month', 'year', 'wod_x']
for col in cls_list:
    df[col] = df[col].astype(str)

In [8]:
def make_line(df, col1):
    fig = px.line(df, x=df.index, y=col1, title=f'{col1}')
    return fig

col_sel1 = pn.widgets.Select(name='single1', options=list(df.columns.values), value='A:総入園者数')
graph = pn.bind(make_line, df, col_sel1)
pn.Column(
    col_sel1,
    graph
)
## 観測
# 入園者数、地下鉄乗車数、地下道歩行数、気温

Column
    [0] Select(name='single1', options=['A:総入園者数', 'B:有料入園者数', ...], value='A:総入園者数')
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)

In [9]:
def make_plot(df, col1, col2, col3):
    fig = px.scatter(df, x=col1, y=col2, title=f'{col1}_{col2}', color=col3)
    return fig

col_sel1 = pn.widgets.Select(name='col1', options=list(df.columns.values), value='A:総入園者数')
col_sel2 = pn.widgets.Select(name='col2', options=list(df.columns.values), value='A:総入園者数')
col_sel3 = pn.widgets.Select(name='col3', options=cls_list)
graph = pn.bind(make_plot, df, col_sel1, col_sel2, col_sel3)


select = pn.Row(col_sel1, col_sel2, col_sel3)
inter = pn.Column(select, graph)
inter

Column
    [0] Row
        [0] Select(name='col1', options=['A:総入園者数', 'B:有料入園者数', ...], value='A:総入園者数')
        [1] Select(name='col2', options=['A:総入園者数', 'B:有料入園者数', ...], value='A:総入園者数')
        [2] Select(name='col3', options=['month', 'year', ...], value='month')
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)

In [7]:
for col in cls_list:
    df[col] = df[col].astype(int)

df6500 = df[df['A:総入園者数'] < 6500]
num = 5
select_list = list()
cols = list(df.columns)
int_values = {
    1: 'J1',
    2: 'さっぽろ（南北線）',
    3: '気温(℃)',
    4: 'A:総入園者数',
    5: 'year'
}
for k, v in int_values.items():
    row = pn.widgets.Select(name=f'selector_{k}', options=cols, value=v)
    select_list.append(row)

def update_para(df, *args):
    fig = px.parallel_coordinates(df, color=args[-1],
                              dimensions=args,
                              color_continuous_scale=px.colors.sequential.Bluered
                              )
    return fig

interactive_para = pn.bind(update_para, df6500, *select_list)
pn.Column(
    pn.Column(*select_list),
    interactive_para
)

Column
    [0] Column
        [0] Select(name='selector_1', options=['A:総入園者数', 'B:有料入園者数', ...], value='J1')
        [1] Select(name='selector_2', options=['A:総入園者数', 'B:有料入園者数', ...], value='さっぽろ（南北線）')
        [2] Select(name='selector_3', options=['A:総入園者数', 'B:有料入園者数', ...], value='気温(℃)')
        [3] Select(name='selector_4', options=['A:総入園者数', 'B:有料入園者数', ...], value='A:総入園者数')
        [4] Select(name='selector_5', options=['A:総入園者数', 'B:有料入園者数', ...], value='year')
    [1] ParamFunction(function, _pane=Plotly, defer_load=False)